In [1]:
import pandas as pd
import json
from scipy.stats import ttest_ind

In [ ]:
# Load baseline results from Excel file
baseline_data = pd.read_excel('inference_timing.xlsx')

# Load your results from JSON file
with open('inference_timing.json', 'r') as json_file:
    your_data = json.load(json_file)

your_data_df = pd.DataFrame(your_data['timing_data'])

# Map the column names between baseline and JSON data
columns_mapping = {
    "sample_time": "sample_time",
    "inference_time": "inference_time",
    "mesh_generation_time": "mesh_generation_time"
}

# Significance level
alpha = 0.05

# Results dictionary
results = {}

# Perform t-tests for each metric
for json_col, baseline_col in columns_mapping.items():
    print(f"\nAnalyzing {json_col}...")

    # Extract and clean data
    baseline_times = pd.to_numeric(baseline_data[baseline_col], errors='coerce').dropna()
    json_times = pd.to_numeric(your_data_df[json_col], errors='coerce').dropna()

    print(f"Baseline Mean: {baseline_times.mean()}, Std: {baseline_times.std()}")
    print(f"Your Data Mean: {json_times.mean()}, Std: {json_times.std()}")

    # Check for valid data
    if baseline_times.std() == 0 or json_times.std() == 0 or len(baseline_times) < 2 or len(json_times) < 2:
        print(f"Skipping t-test for {json_col} due to insufficient data or constant values.")
        results[json_col] = {
            "t_statistic": None,
            "p_value": None,
            "significant_difference": False
        }
        continue

    # Perform t-test
    t_stat, p_value = ttest_ind(json_times, baseline_times, equal_var=False)
    results[json_col] = {
        "t_statistic": t_stat,
        "p_value": p_value,
        "significant_difference": p_value < alpha
    }

# Print results
for col, res in results.items():
    print(f"\n{col}:")
    if res["t_statistic"] is None:
        print("  T-test not performed due to insufficient data or constant values.")
    else:
        print(f"  T-statistic: {res['t_statistic']}")
        print(f"  P-value: {res['p_value']}")
        print(f"  Significant difference: {res['significant_difference']}")


Analyzing sample_time...
Baseline Mean: 2.812862911370025, Std: 0.7428865961577055
Your Data Mean: 2.7330994928199654, Std: 0.7937128535630513

Analyzing inference_time...
Baseline Mean: 0.07447925105007011, Std: 0.1446160079151
Your Data Mean: 0.06251104140992539, Std: 0.03815852609648939

Analyzing mesh_generation_time...
Baseline Mean: 2.7346324753398905, Std: 0.7268126292328049
Your Data Mean: 2.667284468170219, Std: 0.7921096618258832

sample_time:
  T-statistic: -0.7337035724670911
  P-value: 0.46400107837718985
  Significant difference: False

inference_time:
  T-statistic: -0.8001981206582595
  P-value: 0.4252792808036442
  Significant difference: False

mesh_generation_time:
  T-statistic: -0.6264743022725087
  P-value: 0.5317307932202409
  Significant difference: False
